In [ ]:
#
from xgboost import XGBRegressor as XGBR
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import pickle

# evaluation index
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

In [ ]:
# Import dataset
dataset = pd.read_csv('/Users/fuyang/Desktop/chapter2_all_data.csv', index_col=0)
dataset.head()

In [ ]:
# Partition data set
Y = dataset.Score
X = dataset.drop('Score', axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=7)

In [ ]:
# Maximum depth of tree and minimum leaf node sample weight
param_test1 = {'max_depth':range(3,10,2),'min_child_weight':range(2,7,2)}
 
gsearch1 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=140, max_depth=5,
                                         min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test1, scoring='r2',n_jobs=4, cv=5)
gsearch1.fit(X_train,Y_train)
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
# n_estimators
param_test2 = {'n_estimators': [10, 20, 50, 100, 150, 200, 250, 300, 400, 500]}

gsearch2 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=140, max_depth=5,
                                         min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test2, scoring='r2',n_jobs=4, cv=5)
gsearch2.fit(X_train,Y_train)
gsearch2.best_params_, gsearch2.best_score_

In [ ]:
param_test3 = {'gamma':[i/100.0 for i in range(0,100)]}

gsearch3 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test3, scoring='r2',n_jobs=4, cv=5)
gsearch3.fit(X_train,Y_train)
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {'subsample':[i/10.0 for i in range(1,10)],'colsample_bytree':[i/10.0 for i in range(1,10)]}

gsearch4 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0.92, subsample=0.8, colsample_bytree=0.8, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test4, scoring='r2',n_jobs=4, cv=5)
gsearch4.fit(X_train,Y_train)
gsearch4.best_params_, gsearch4.best_score_

In [ ]:
# gamma 
param_test5 = {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}

gsearch5 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0.92, subsample=0.8, colsample_bytree=0.8, 
                                         reg_alpha=0, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test5, scoring='r2',n_jobs=4, cv=5)
gsearch5.fit(X_train,Y_train)
gsearch5.best_params_, gsearch5.best_score_

In [ ]:
#learning_rate
param_test6 = {'learning_rate':[0, 0.001, 0.005, 0.01, 0.05,0.1,0.5,1]}

gsearch6 = GridSearchCV(estimator =XGBR( learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0.92, subsample=0.8, colsample_bytree=0.8, 
                                         reg_alpha=0, objective= 'reg:linear',
                                         nthread=4, scale_pos_weight=1, seed=27),
                                         param_grid = param_test6, scoring='r2',n_jobs=4, cv=5)
gsearch6.fit(X_train,Y_train)
gsearch6.best_params_, gsearch6.best_score_

In [ ]:
# 
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

model = XGBR(learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0.92, subsample=0.8, colsample_bytree=0.8, 
                                         reg_alpha=0.1)
model.fit(rescaledX, Y_train)
score = cross_val_score(model, rescaledX, Y_train, cv=5, scoring=scoring2)
print(score.mean(), "+/-", score.std())

In [ ]:
# Performance of the model on the XGBoost test set and training set
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

model = XGBR(learning_rate =0.1, n_estimators=50, max_depth=5,
                                         min_child_weight=6, gamma=0.92, subsample=0.8, colsample_bytree=0.8, 
                                         reg_alpha=0.1)
model.fit(rescaledX, Y_train)

# Evaluate the training set
rescaledX_train = scaler.transform(X_train)
pre_train = model.predict(rescaledX_train)
print('Training_data，MSE：%s' % (mean_squared_error(Y_train, pre_train)))
print('Training_data，MAE：%s' % (mean_absolute_error(Y_train, pre_train)))
print('Training_data，R2：%s' %  (r2_score(Y_train, pre_train)))

# Evaluation test set
rescaledX_test = scaler.transform(X_test)
pre_test = model.predict(rescaledX_test)
print('Testing_data，MSE：%s' % (mean_squared_error(Y_test, pre_test)))
print('Testing_data，MAE：%s' % (mean_absolute_error(Y_test, pre_test)))
print('Testing_data，R2：%s' % (r2_score(Y_test, pre_test)))